In [ ]:
# Librerías
import random  # combinaciones aleatorias
import unicodedata  # normalizar acentos



# Palabras de terror escogidas
terror_words = [
    "espectro", "cripta", "ataúd", "tinieblas", "cementerio", "maldición",
    "fantasma", "exorcismo", "calavera", "posesión", "hechizo", "bruja",
    "ultratumba", "demonio", "sepulcro"
]


# Diccionario de frases de terror
terror_phrases = {
    "bienvenida": [
        "Bienvenida... al último juego de tu vida.",
        "La noche es oscura. Bienvenida, esta puede ser la última palabra que adivines.",
        "Has entrado. ¡Bienvenida! No podrás salir hasta que ganes."
    ],
    "acierto": [
        "Sí... pero ellos ya saben que estas aquí.",
        "Acertaste... pero algo se mueve en las sombras.",
        "Muy bien... por ahora sobrevives."
    ],
    "fallo": [
        "Has fallado... y la soga aprieta un poco más.",
        "No... y algo se arrastra hacia tí.",
        "Ese error te acerca al final."
    ],
    "victoria": [
        "Has sobrevivido... pero no por mucho tiempo.",
        "Ganaste... o eso crees.",
        "La palabra era tu salvación... ésta vez."
    ],
    "derrota": [
        "Has perdido... y ahora eres uno de ellos.",
        "No lograste escapar... la oscuridad te envuelve.",
        "Caiste... y nadie vendrá a ayudarte."
    ]
}


# Selección aleatoria de palabra
word = random.choice(terror_words)
word = unicodedata.normalize('NFKD', word).encode('ASCII', 'ignore').decode('utf-8')
print(word) #PARA PROBAR QUE NUESTRO JUEGO FUNCIONA CORRECTAMENTE


# Mensajes iniciales
print(random.choice(terror_phrases["bienvenida"]))
print(f"Tu palabra contiene {len(word)} letras: {'_ ' * len(word)}")


# Etapas del juego del ahorcado
hangman = [
    """
     ------
     |    |
     |
     |
     |
    _|_
    """,
    """
     ------
     |    |
     |    O
     |
     |
    _|_
    """,
    """
     ------
     |    |
     |    O
     |    |
     |
    _|_
    """,
    """
     ------
     |    |
     |    O
     |   /|
     |
    _|_
    """,
    """
     ------
     |    |
     |    O
     |   /|\\
     |
    _|_
    """,
    """
     ------
     |    |
     |    O
     |   /|\\
     |   /
    _|_
    """,
    """
     ------
     |    |
     |    O
     |   /|\\
     |   / \\
    _|_
    """
]

# Variables de control del juego
correct_letters = set()
used_letters = set()
attempts_per_player = len(hangman) - 1  # intentos máximos por cada jugador. Variable para poder diferenciar los intentos de uno y de otro jugador
players = ["JUGADOR 1", "JUGADOR 2"]
score = {players[0]: 0, players[1]: 0} #marcador simple. 1 punto al acertar la palabra entera o cuando el otro agota todos los intentos
turn = 0  # empieza el Jugador 1
mistakes = {players[0]: 0, players[1]: 0}  # errores por jugador. Lo ponemos en forma de diccionario para que cuente los errores de ambos jugadores

print(f"Tienes {attempts_per_player} intentos para acertar la palabra. Vivir o morir. Tú decides.")

# Bucle principal del juego
while True:
    print("\n" + "-"*50)  # separador de turnos
    current_player = players[turn] # el jugador actual
    print(f"\nTURNO DE {current_player}")
    print(hangman[mistakes[current_player]])  # dibujo según errores de este jugador

    # Mostrar palabra parcialmente  (la letra acertada o un guión)
    incomplete_word = ""
    for letter_in_word in word:
        if letter_in_word.lower() in correct_letters:
            print(letter_in_word, end=" ")
            incomplete_word += letter_in_word + " "
        else:
            print("_", end=" ")
            incomplete_word += "_ "
    print()
    print(f"Palabra actual: {incomplete_word.strip()}")
    print(f"Letras usadas: {', '.join(sorted(used_letters)) if used_letters else 'Ninguna'}")
    print(f"A {current_player} le quedan {attempts_per_player - mistakes[current_player]} intentos") #intentos que quedan por jugador. Muestra los del Inicio de la ronda actual

    # Entrada del jugador o salir si quiere interrumpir el juego. Evitamos con salir darle a interrumpir manualmente
    letter = input("Introduce una letra (o escribe 'salir' para terminar el juego): ").strip().lower()

    if letter == "salir":
        print("Has salido del juego")
        print(f"La palabra era: {word}")
        break  #salimos del juego al escribir salir

    # Normalizar acento letra
    letter = unicodedata.normalize('NFKD', letter).encode('ASCII', 'ignore').decode('utf-8')

    # Validaciones
    if len(letter) != 1 or not letter.isalpha():
        print(f"{letter} no es un caracter válido. Prueba de nuevo.")
        continue
    if letter in used_letters:
        print(f"Ya has usado la letra '{letter}'. Prueba con otra.")
        continue

    used_letters.add(letter)

    # Comprobación letra en palabra
    if letter in word.lower():
        correct_letters.add(letter)
        print(f"{random.choice(terror_phrases['acierto'])}. La '{letter}' está en la palabra buscada.")
        
    else:
        mistakes[current_player] += 1 #si letra no está palabra, sumamos fallo
        print(f"{random.choice(terror_phrases['fallo'])}. La '{letter}' no está en la palabra buscada.")
        failed_player = current_player  # Variable que guarda quién ha fallado
        turn = 1 - turn  # cambia de turno sólo si hay fallo

        # Condición de derrota
        if mistakes[failed_player] == attempts_per_player:
            print(hangman[attempts_per_player]) #tenemos que ponerlo aquí para que nos salga la última pierna del ahorcado antes de decir que ha perdido
            print(f"{random.choice(terror_phrases['derrota'])}")
            print(f"La palabra era: {word}")
            winner = winner = players[0] if failed_player == players[1] else players[1]  #si el que falla, agota todos los intentos, automáticamente gana el otro
            print(f"¡{winner} ES EL GANADOR DEL JUEGO DEL AHORCADO!")
            score[winner] += 1 #marcador que se suma al ganador
            print(f"El Marcador Final es: {players[0]}: {score[players[0]]} | {players[1]}: {score[players[1]]}")
            break

    # Condición de victoria
    if correct_letters >= set(word.lower()):
        print(random.choice(terror_phrases["victoria"]))
        print(f"¡{current_player} ERES EL GANADOR DEL JUEGO DEL AHORCADO!")
        print(f"La palabra era: {word}")
        score[current_player] += 1
        print(f"Marcador actual: {players[0]}: {score[players[0]]} | {players[1]}: {score[players[1]]}")
        break




Sí... pero ellos ya saben que estas aquí.. La 's' está en la palabra buscada.

--------------------------------------------------

TURNO DE JUGADOR 1

     ------
     |    |
     |
     |
     |
    _|_
    
s _ _ _ _ _ _ _ 
Palabra actual: s _ _ _ _ _ _ _
Letras usadas: s
A JUGADOR 1 le quedan 6 intentos
No... y algo se arrastra hacia tí.. La 'd' no está en la palabra buscada.

--------------------------------------------------

TURNO DE JUGADOR 2

     ------
     |    |
     |
     |
     |
    _|_
    
s _ _ _ _ _ _ _ 
Palabra actual: s _ _ _ _ _ _ _
Letras usadas: d, s
A JUGADOR 2 le quedan 6 intentos
Muy bien... por ahora sobrevives.. La 'p' está en la palabra buscada.

--------------------------------------------------

TURNO DE JUGADOR 2

     ------
     |    |
     |
     |
     |
    _|_
    
s _ p _ _ _ _ _ 
Palabra actual: s _ p _ _ _ _ _
Letras usadas: d, p, s
A JUGADOR 2 le quedan 6 intentos
Acertaste... pero algo se mueve en las sombras.. La 'l' está en la palabra busc